In [1]:
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import numpy as np
from scipy.optimize import minimize
import pandas as pd
from scipy.stats import poisson, uniform, expon, pareto
from tqdm import tqdm
from mdptoolbox import mdp, util
import itertools
from scipy.sparse import csr_matrix, lil_matrix
from matplotlib.patches import Patch
import math
import random
import sympy as sp
from sympy.printing.latex import print_latex

# sim for avg. block length
- sim the markov chain
- for state 2, check that the average amount of time until the next attacker block is 

$\alpha \cdot \lambda + (1-\alpha)\alpha\cdot2\lambda $

In [6]:
alpha, beta, gamma = 0.3, 1., 0.0

In [18]:
# 0 = 0, 0' = 1, 0'' = 2, 1 = 3, 2 = 4, ... 
markov = np.zeros(shape=(50,50))

# state 0
markov[0,0] = (1-alpha) + alpha*np.exp(-beta)
markov[0,3] = alpha*(1-np.exp(-beta))

# state 0'
markov[1,0] = 1

# state 0''
markov[2,0] = 1

# state 1
markov[3,1] = 1-alpha
markov[3,4] = alpha

# state 2
markov[4,2] = 1-alpha
markov[4,5] = alpha

for i in range(5,49):
    markov[i,i-1] = 1-alpha
    markov[i,i+1] = alpha

np.sum(markov, axis=1)

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0.])

In [178]:
states   = np.arange(50)
curState = 0
transcript = []
blkTimes   = []
zeroPrimes = {}
zeroP2s    = {}
for i in tqdm(range(int(1e6))):
    
    nextState = np.random.choice(states, p=markov[curState])
    blockTime = np.random.exponential()
    transcript.append(nextState)
    blkTimes.append(blockTime)

    if curState == 1: # 0'
        assert(nextState == 0)
        attBlock     = np.random.binomial(1, alpha)
        blockTime    = np.random.exponential()
        zeroPrimes[i] = [attBlock, blockTime]
    if curState == 2: # 0''
        assert(nextState == 0)
        attBlock   = np.random.binomial(1, alpha)
        blockTime  = np.random.exponential()
        zeroP2s[i] = [attBlock, blockTime]       
    curState = nextState
    
transcript = np.asarray(transcript)
blkTimes = np.asarray(blkTimes)
transcript

100%|██████████| 1000000/1000000 [00:17<00:00, 58745.79it/s]


array([0, 0, 0, ..., 0, 3, 4])

In [179]:
times = []
for i, val in enumerate(transcript[:-2]):
    if transcript[i] == 4 and transcript[i-1] == 3:
        if transcript[i+1] == 5:
            times.append(blkTimes[i+1])
        else:
            assert(transcript[i+1] == 2)
            attBlock, blockTime = zeroP2s[i+2]
            if attBlock:
                times.append(blkTimes[i+1] + blockTime)
            else:
                times.append(0)
times = np.array(times)
np.mean(times)

0.7211903040442489

In [172]:
alpha + (1-alpha)*alpha*2

0.72

In [126]:
np.mean(np.random.exponential(size=10000))

0.9987869497789296

In [102]:
zeroP2s[6]

[1, 0.018453481021762034]

```
0 = 0
1 = 0'
2 = 0''
3 = 1
4 = 2
5 = 3
```

In [ ]:
def sim(alpha, gamma, beta, n=int(1e5)):
    state      = [0,0] # (attacker, honest)
    transcript = []
    times      = []
    for _ in range(n):
        attBlock  = np.random.binomial(1, alpha)
        blockTime = np.random.exponential()

        if state == [0,0]:
            if attBlock:
                if execRews < beta:
                    state = [1,0]
                    transcript.append(1)
                    times.append(blockTime)
                else:
                    transcript.append(0)
                    times.append(blockTime)
            else:
                transcript.append(0)
                times.append(blockTime)
            continue

        if state == [1,0]:
            if attBlock:
                state = [2,0]
                transcript.append(2)
                times.append(blockTime)
            # 0' state
            else:
                transcript.append(-1)
                times.append(blockTime)
                sample         = np.random.uniform()
                nextBlockTime  = np.random.exponential()
                if sample < alpha:
                    pass
                elif sample <= alpha + ((1-alpha)*gamma):
                    pass
                else:
                    pass
                state = [0,0]
                transcript.append(0)
                times.append(nextBlockTime)
            continue

        if attBlock:
            state[0] += 1
            transcript.append(state[0]-state[1])
            times.append(blockTime)
        else:
            state[1] += 1
            transcript.append(state[0]-state[1])
            times.append(blockTime)
            
        

        diff = state[0] - state[1]
        if diff == 1:
            # 0'' state
            attBlock  = np.random.binomial(1, alpha)
            blockTime = np.random.exponential()
            if attBlock:
                pass
            else:
                pass
            state = [0,0]
    return